In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

2023-04-09 17:09:13.030187: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# TODO
- linear interpolation
- normalise for each person separately
- join into a single csv and save

In [101]:
# for i in range(1,4):


def interpolate_aggr(id, path="data/aggregated_individual_data_interpolation/", save_path="data/aggregated_individual_data_interpolation"):
    """Use time-interpolation on dataset to fill np.NaN values. Beginning and end are filled with 0s."""
    df = pd.read_csv(f"{path}/raw/{id}_aggregated.csv", parse_dates=["date"], index_col="date")
    df.drop('Unnamed: 0', axis=1, inplace=True)
    df.interpolate(inplace=True, method="time")
    df.reset_index(inplace=True)
    df['days'] = (df['date'] - df['date'].min()).dt.days.astype(int)
    df.fillna(0) # fill beginning and end with 0 if cannot interpolate
    df.to_csv(f"{path}/interpolation/{id}_interpolated.csv")

for id in range(27):
    interpolate_aggr(id)


OSError: Cannot save file into a non-existent directory: 'data/aggregated_individual_data_interpolation/interpolation'

In [ ]:

# df.drop(["date", "circumplex.arousal_std","circumplex.arousal_std","circumplex.valence_std"],inplace=True, axis=1)

In [ ]:
plt.plot(df["days"], df["mood"])